In [1]:
import importlib
import full_model
import numpy as np

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 21.47 sec


In [3]:
data_loaded = full_model.loadmat("./saved/processed/yaleB.mat")
images_norm = data_loaded['X']
labels = data_loaded['Y'].reshape(-1)

In [11]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 3, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 3, name='maxIter2'),]

In [12]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    global ep_prtr_, ep_, seed_
    try:
        #return 1-full_model.run_model(images_norm, labels, seed_, ep_prtr_, ep_, **params)[0]
        return params['lr_pretrain']*params['lr']*params['alpha1']*params['alpha2']
    except Exception as ex:
        if(type(ex) == KeyError):
            raise ex
        print("Caught a " + type(ex).__name__ + ". Returning 1.0")
        return 1

In [13]:
from skopt import gp_minimize, dummy_minimize, forest_minimize, gbrt_minimize
import time

def optimize(function, iterations, epochs_pretrain, epochs, random_seed):
    start_time = time.time()
    
    # global b/c I couldn't find a better way to directly pass these to the objective function
    global ep_prtr_, ep_, seed_
    ep_prtr_ = epochs_pretrain
    ep_ = epochs
    seed_ = random_seed
    
    # kwargs b/c dummy_minimize can not take n_jobs
    params = {"n_calls":iterations, "random_state":random_seed, "verbose":True}
    if(function != dummy_minimize):
        params["n_jobs"] = -1
    result = function(objective, space, **params)
    opt_stats(result, start_time)
    return result

In [14]:
from skopt.plots import plot_convergence

def opt_stats(res, start_time=None):
    plot_convergence(res, yscale="log")
    print("\n------------------")
    for i in range(10, len(res.func_vals)+1, 10):
        print("{0:d}: {1:.4f}".format(i, min(res.func_vals[0:i])))
    print("Best score: {0:.4f}".format(res.fun))
    print("Best parameters: {0:}".format(res.x))
    if(start_time is not None):
        print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))

Reloading functions:

In [84]:
from copy import deepcopy
from skopt.optimizer import base_minimize

def reload_bad(result, addtl_calls):
    args = deepcopy(result.specs['args'])
    args['n_calls'] = addtl_calls
    args['n_random_starts'] = 0
    args['x0'] = deepcopy(result.x_iters)
    args['y0'] = deepcopy(result.func_vals)
    args['random_state'] = deepcopy(result.random_state)
    return base_minimize(**args)

In [85]:
from copy import deepcopy
from skopt.learning import GaussianProcessRegressor
from skopt.optimizer import base_minimize
from sklearn.utils import check_random_state
import warnings

def func_new(params):
    global func_, xs_, ys_
    if(len(xs_) > 0):
        y = ys_.pop(0)
        if(params != xs_.pop(0)):
            warnings.warn("Deviated from expected value, re-evaluating", RuntimeWarning)
        else:
            return y
    return func_(params)

def reload(result, addtl_calls, init_seed=None):
    args = deepcopy(result.specs['args'])
    args['n_calls'] += addtl_calls
    
    # global b/c I couldn't find a better way to pass
    global func_, xs_, ys_
    func_ = args['func']
    xs_ = list(result.x_iters)
    ys_ = list(result.func_vals)
    args['func'] = func_new
    
    # recover initial random_state
    if(isinstance(args['random_state'], np.random.RandomState)):
        args['random_state'] = check_random_state(init_seed)
        # if gp_minimize
        if(isinstance(result.specs['args']['base_estimator'], GaussianProcessRegressor)):
            args['random_state'].randint(0, np.iinfo(np.int32).max)
    
    # run the optimization
    result_new = base_minimize(**args)
    
    # change the function back, to reload multiple times
    result_new.specs['args']['func'] = func_
    
    return result_new

In [10]:
tests = {'fun', 'space', 'x', 'func_vals', 'x_iters'}
def test(val1, val2):
    assert np.alltrue([np.alltrue(val1[key] == val2[key]) for key in tests]), "TESTS FAILED"
    print("ALL TESTS PASSED!")

In [ ]:
used_func = forest_minimize
seed = 1234
res1 = optimize(used_func, 12, 1, 1, seed)
res1_copy = deepcopy(res1)
res12 = reload(res1, 4, seed)
res23 = reload(res12, 4, seed)
res13 = reload(res1, 8, seed)
res3 = optimize(used_func, 20, 1, 1, seed)

from skopt import dump, load
dump(res1, "temp.txt")
res1_loaded = load("temp.txt")
res12_loaded = reload(res1_loaded, 4, seed)

In [97]:
test(res13, res3)
test(res23, res3)
test(res1, res1_loaded)
test(res12, res12_loaded)
test(res1_copy, res1)

ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
ALL TESTS PASSED!
